In [ ]:
# The file appears to have a structure where each dataset is preceded by a row containing its name
# First, identify the rows that contain these names
sensor_name_rows = data[0].str.contains('binary_sensor')

# Split the data into separate datasets based on these rows
datasets = {}
current_sensor = None

for i, row in data.iterrows():
    if sensor_name_rows[i]:
        # This row contains the sensor name, which we'll use as the dataset name
        current_sensor = row[0].split('.')[1]  # Extract name part after 'binary_sensor.'
        datasets[current_sensor] = []
    elif current_sensor:
        # Add rows to the current dataset, skip the first row which contains column names
        if row[0] != "Time stamp":
            datasets[current_sensor].append(row)

# Convert lists to dataframes and save them as CSV files
saved_files = []
for sensor_name, dataset in datasets.items():
    df = pd.DataFrame(dataset, columns=['Time stamp', 'State'])
    file_name = f'/mnt/data/{sensor_name}.csv'
    df.to_csv(file_name, index=False)
    saved_files.append(file_name)

saved_files  # List of saved files' paths



In [ ]:
# It appears that the rows were added as Series objects, not as lists. This caused the formatting issue.
# Let's correct this by converting each Series to a list before adding it to the dataset

# Re-creating the datasets with the corrected approach
corrected_datasets = {}
current_sensor = None

for i, row in data.iterrows():
    if sensor_name_rows[i]:
        # This row contains the sensor name
        current_sensor = row[0].split('.')[1]  # Extract name part after 'binary_sensor.'
        corrected_datasets[current_sensor] = []
    elif current_sensor:
        # Add rows as lists to the current dataset, skip the first row which contains column names
        if row[0] != "Time stamp":
            corrected_datasets[current_sensor].append(row.tolist())

# Convert lists to dataframes and re-save them as CSV files
corrected_saved_files = []
for sensor_name, dataset in corrected_datasets.items():
    df = pd.DataFrame(dataset, columns=['Time stamp', 'State'])
    file_name = f'/mnt/data/{sensor_name}_corrected.csv'
    df.to_csv(file_name, index=False)
    corrected_saved_files.append(file_name)

corrected_saved_files  # List of corrected saved files' paths


In [ ]:
# Saving the merged dataframe to a CSV file
merged_file_path = '/mnt/data/merged_sensors_data.csv'
merged_df.to_csv(merged_file_path, index=False)

merged_file_path  # Path to the merged CSV file


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculating the correlation matrix
correlation_matrix = merged_df.iloc[:, 1:].corr()  # Excluding the 'Time stamp' column for correlation

# Plotting the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix Between Sensors")
plt.show()


In [ ]:
# Removing the date substring from 'Time stamp' column and keeping only the time in the format hh:mm:ss
merged_df['Time stamp'] = pd.to_datetime(merged_df['Time stamp']).dt.time

# Summing the rows with the same resulting 'Time stamp'
grouped_df = merged_df.groupby('Time stamp').sum().reset_index()

grouped_df.head()  # Display the first few rows of the modified dataframe
